# Get a list of Targets based on a pathway
#### Create a POST request to the RaMP API

In [278]:
import requests
import pandas as pd
import json

# see https://rampdb.nih.gov/api for details

ramp_url = "https://rampdb.nih.gov/api/analytes-from-pathways"

post_body = {
    "analyte_type": "gene",
    "names_or_ids": "names",
    "max_pathway_size": 500,
    "pathway": ["Sleep regulation"]
}

#### do the POST request

In [279]:
response = requests.post(ramp_url, data=post_body)
response_json = json.loads(response.text)["data"]

#### Parse the result

In [280]:
target_uniprots = []
for entry in response_json:
    ids = entry['sourceAnalyteIDs'].split(',')
    for id in ids:
        if id.startswith('uniprot:'):
            uniprot = id.split(':')[1]
            target_uniprots.append(uniprot)
            
target_uniprots = list(set(target_uniprots))

print(pd.DataFrame(target_uniprots))

              0
0        H7C0Z2
1    A0A7P0TBK2
2        P50129
3        P56645
4    A0A2R8Y567
..          ...
191      B2R727
192      Q16783
193      Q99808
194  A0A1B0GTH5
195  A0A0S2Z3B9

[196 rows x 1 columns]


# Get gene-disease associations and drug activities for those targets
#### Create a POST request to the Pharos API

In [281]:
# see https://pharos.nih.gov/api for details

pharos_url = "https://pharos-api.ncats.io/graphql"

post_body = {
  "query": """query targetDetails($targets: [String]){
  targets(targets: $targets) {
    count
    targets(top: 100000) {
      name
      uniprot
      drugs:ligands(isdrug: true) {
        name
        smiles
        synonyms {
          value
          name
        }
      }
      ligands(isdrug: false) {
        name
        smiles
        synonyms {
          value
          name
        }
      }
      diseases {
        mondoID
        name
        mondoEquivalents {
          id
          name
        }
      }
    }
  }
}""",
  "variables": {"targets": target_uniprots},
  "operationName": "targetDetails"
}

#### do the POST request

In [282]:
response = requests.post(pharos_url, json=post_body)
response_json = json.loads(response.text)["data"]
target_list = response_json["targets"]["targets"]

#### Parse the disease results

In [283]:
disease_objects = [disease for target in target_list for disease in target["diseases"]]
disease_names = list(set(disease["name"] for disease in disease_objects))

print(pd.DataFrame(disease_objects))

gard_aliases = [alias["id"] for disease in disease_objects if disease["mondoEquivalents"] is not None for alias in disease["mondoEquivalents"] if alias["id"].startswith("GARD:")]
gard_aliases = list(set(gard_aliases))

print(gard_aliases)

           mondoID                                      name  \
0    MONDO:0005391                    restless legs syndrome   
1    MONDO:0020560          atypical teratoid rhabdoid tumor   
2    MONDO:0021191                      malignant ependymoma   
3    MONDO:0008170                            ovarian cancer   
4    MONDO:0018177                              glioblastoma   
..             ...                                       ...   
345  MONDO:0005086                      renal cell carcinoma   
346  MONDO:0007154  arteriovenous malformations of the brain   
347  MONDO:0005265                inflammatory bowel disease   
348  MONDO:0006662                        aseptic meningitis   
349  MONDO:0004994                            cardiomyopathy   

                                      mondoEquivalents  
0    [{'id': 'DOID:0050425', 'name': 'restless legs...  
1    [{'id': 'DOID:2129', 'name': 'atypical teratoi...  
2    [{'id': 'DOID:5074', 'name': 'malignant ependy...  
3  

#### Parse the drug results

In [284]:
drug_objects = [drug for target in target_list for drug in target["ligands"]]
drug_objects.extend([drug for target in target_list for drug in target["drugs"]])

drug_names = list(set(drug["name"] for drug in drug_objects))

print(pd.DataFrame(drug_objects))

unii_aliases = [synonym["value"] for drug in drug_objects for synonym in drug["synonyms"] if synonym["name"] == "unii"]
unii_aliases = list(set(unii_aliases))
print(unii_aliases)

                                                  name  \
0    3-[[5-[4-(tert-butylsulfamoyl)-1-naphthyl]-4-(...   
1                  2-(4-tert-Butyl)phenoxycyclohexanol   
2    (2S,5S,8S,11S,14S,17S,20S,23S,26S,29S,32S,35S,...   
3    (2S,5S,8S,11S,14S,17S,20S,23S,26S,29S,32S,35S,...   
4    (2S,5S,8S,11S,14S,17S,20S,23S,26S,29S,32S,35S,...   
..                                                 ...   
282                                       fluphenazine   
283                                           lisuride   
284                                         metirosine   
285                                       prednisolone   
286                                         siltuximab   

                                                smiles  \
0    CC(C)(C)NS(=O)(=O)c1ccc(-c2sc(C(=O)N[C@H]3C[C@...   
1                         CC(C)(C)c1ccc(OC2CCCCC2O)cc1   
2    CC[C@H](C)[C@H](NC(=O)[C@@H]1CCCN1C(=O)[C@@H](...   
3    CC[C@H](C)[C@H](N)C(=O)N[C@@H](CCCCN)C(=O)N1CC...   
4    CC[C@H](

## Get Disease Details
#### Create POST request to the RDAS API

In [285]:
# see https://rdas.ncats.nih.gov/api/diseases for details

rdas_url = "https://rdas.ncats.nih.gov/api/diseases"

post_body = {
    "query": """query diseaseQuery($where: GARDWhere) {
  gards(where: $where) {
    GardId
    GardName
    MeSH
    Synonyms
    COUNT_ARTICLES
    COUNT_TRIALS
    ClassificationLevel
  }
}""", 

  "variables": {"where": {"GardId_IN": gard_aliases}},
  "operationName": "diseaseQuery"
}

#### do the POST request

In [286]:
response = requests.post(rdas_url, json=post_body)
response_json = json.loads(response.text)["data"]

#### parse the results

In [287]:
gard_data = [{
    "GardId": obj["GardId"],
    "GardName": obj["GardName"],
    "COUNT_ARTICLES": obj["COUNT_ARTICLES"],
    "COUNT_TRIALS": obj["COUNT_TRIALS"],
    "ClassificationLevel": obj["ClassificationLevel"]
} for obj in response_json["gards"]]
df = pd.DataFrame(gard_data)
print(df)

          GardId                                           GardName  \
0   GARD:0000555              X-linked adrenal hypoplasia congenita   
1   GARD:0007015                                         Meningioma   
2   GARD:0006383                             Carcinoma of esophagus   
3   GARD:0009953                                  Oligodendroglioma   
4   GARD:0006105                           Chronic myeloid leukemia   
5   GARD:0007392      Behavioral variant of frontotemporal dementia   
6   GARD:0006855                           Landau-Kleffner syndrome   
7   GARD:0007185                                      Neuroblastoma   
8   GARD:0005748  Severe combined immunodeficiency due to adenos...   
9   GARD:0002491                                       Glioblastoma   
10  GARD:0007162                                  Narcolepsy type 1   
11  GARD:0009371                                Pituitary carcinoma   
12  GARD:0007513                  Pyruvate dehydrogenase deficiency   
13  GA

## Get Drug Details
#### Create GET request strings

In [ ]:
drug_requests = [f"https://drugs.ncats.io/api/v1/substances({unii})?view=full" for unii in unii_aliases]

drug_objects = []
for req in drug_requests:
    
    drug_json = json.loads(requests.get(req).text)
    
    if "_name" in drug_json:
        drug_obj = {
            "name": drug_json["_name"], 
            "approval_ID": drug_json["approvalID"],
            "substanceClass": drug_json["substanceClass"],
            "status": drug_json["status"]
        }
        if "structure" in drug_json:
            drug_obj["formula"] = drug_json["structure"]["formula"]
            drug_obj["mwt"] = drug_json["structure"]["mwt"]
        
        drug_objects.append(drug_obj)
    else:
        print(drug_json)

df = pd.DataFrame(drug_objects)
print(df)